In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
def plot_network(G):
    plt.figure(figsize=(10, 15))
    pos = nx.spring_layout(G,k=0.6)  
    nx.draw_networkx_nodes(G, pos, node_size=50, node_color='blue')
    nx.draw_networkx_edges(G, pos, alpha=0.5)
    label_pos = {key: (pos[key][0], pos[key][1] + 0.025) for key in pos}
    nx.draw_networkx_labels(G, label_pos, font_size=8, verticalalignment='bottom')
    plt.title('Gene Coexpression Network')
    plt.show()

Load in data and create adata object

In [ ]:
file_path = '/scRNAseq-TopAnalysis/Data/scRNA-seq-data/Alzheimers/GSE103334.txt'
chunksize = 500  

chunks = []
for chunk in pd.read_csv(file_path, sep='\t', index_col=0, header=0, chunksize=chunksize):
    chunks.append(chunk)
X = pd.concat(chunks, axis=0)
print('Gene Expression Size:', X.shape)

# Filter out genes with low variance
# Set values below 1e-6 to 0
X[X < 1e-6] = 0
row_variances = np.var(X, axis=1)
top_indices = np.argsort(row_variances)[-2000:]
X = X.iloc[top_indices, :]
print('Post gene filtering X shape:', X.shape)

In [ ]:
# create adata object
gene_names = X.index.values
expression_data = X.values.astype(float).T
adata = sc.AnnData(X=expression_data)
adata.var_names = gene_names
adata.obs_names = X.columns

In [ ]:
# name each sample according to its temporal label

def simplify_obs_name(name):
    parts = name.split('_')
    if len(parts) >= 2:
        new_name = '_'.join(parts[:2])
    else:
        new_name = name
    return new_name

obs_names_series = pd.Series(adata.obs_names, index=adata.obs_names)
simplified_names = obs_names_series.apply(simplify_obs_name)
adata.obs['temporal'] = simplified_names.values

In [ ]:
# data preprocessing and tsne visualization
adata2 = adata.copy() # copy adata so as not to change the original object

sc.pp.normalize_total(adata2)
sc.pp.log1p(adata2)
sc.pp.pca(adata2)
sc.pp.neighbors(adata2, n_neighbors=10)

sc.tl.tsne(adata2)

In [ ]:
# add temporal labels to original adata
adata.obs['temporal'] = adata2.obs['temporal']
adata.obs['temporal'] = adata.obs['temporal'].astype('category')

In [ ]:
sc.pl.tsne(adata2, color='temporal')

In [ ]:
# collect control group together into one label for convenience
def assign_label(name):
    if name.startswith('CK_'):
        return 'CK'
    elif name.startswith('CKp25_'):
        parts = name.split('_')
        if len(parts) >= 2:
            week = parts[1]
            return week 
        else:
            return 'CKp25'  
    else:
        return 'Unknown'  
    
obs_names_series = pd.Series(adata.obs_names, index=adata.obs_names)
adata2.obs['grouped_temporal'] = obs_names_series.apply(assign_label).values

adata.obs['grouped_temporal'] = obs_names_series.apply(assign_label).values # add these new labels to original adata as well
adata.obs['grouped_temporal'] = adata.obs['grouped_temporal'].astype('category')

sc.pl.tsne(adata2, color='grouped_temporal')

Construct GCN for different stages of progression

In [ ]:
from GeneCoexpressionNetworks import GeneCoexpressionNetwork

# construct gene coexpression network
adata3 = adata.copy()
threshold=0.3 # early and progressive stages
#threshold=0.4 # late stage
gcn = GeneCoexpressionNetwork(adata3, threshold, 75, '1w', 'grouped_temporal')
gene_names, G = gcn.GCN()

In [ ]:
plot_network(G)

Calculate topological significance of each differentially expressed gene and intersect scores across each scale

In [ ]:
from SignificantGenes import TopologicalSignificancesParallelComputation
# compute topological significances of each gene
#radii = [0.4,0.5,0.6,0.7] # late stage
radii = [0.3,0.35,0.4,0.45,0.5] # early and progressive stages
radii = 1 / np.array(radii)
radii = radii[::-1]

args = [(G, gene, radii) for gene in gene_names]

scores = TopologicalSignificancesParallelComputation(args)

In [ ]:
from SignificantGenes import intersect_top_genes
genes = intersect_top_genes(scores, gene_names, 20)
print(genes)

Pathway enrichment analysis

In [ ]:
import gseapy as gp
enr=gp.enrichr(gene_list=genes,  
               gene_sets=['KEGG_2021_Human'], 
               organism='human', 
               outdir=None,
               cutoff=0.05)

In [ ]:
results = enr.results
results.head()

In [ ]:
significant_pathways = results[results['Adjusted P-value'] < 0.05]
#significant_pathways = significant_pathways[significant_pathways['Combined Score'] > 250]
significant_pathways.shape

In [ ]:
# Plotting
plt.figure(figsize=(10, 6))
plt.barh(significant_pathways['Term'], significant_pathways['Combined Score'], color='skyblue')
plt.xlabel('Combined Score', fontsize=30)
plt.title('Pathway Analysis (Early)', fontsize=30)
plt.tick_params(axis='both', which='major', labelsize=30)
plt.gca().invert_yaxis()  # Highest scores on top
plt.show()